In [ ]:
from   collections import defaultdict
import hashlib
import re

from   datalabs.access.aws import AWSClient
import pandas

In [ ]:
sheet = pandas.read_excel("snomed_cpt_map_fsn_descr_20230131.xlsx")

## Transformer

## Clean Up Values

In [ ]:
snomed = sheet.loc[:, ["Concept Id", "FSN", "Map Category", "CPT Code", "CPT Descriptor", "Map Id"]]

snomed.loc[:, "Concept Id"] = snomed["Concept Id"].fillna(method='ffill').astype('int').astype('str')
snomed.loc[:, "FSN"] = snomed["FSN"].fillna(method='ffill')
snomed.loc[:, "FSN"] = snomed["FSN"].astype('str')
snomed.loc[snomed["FSN"] == "nan", "FSN"] = ""

snomed.loc[:, "CPT Descriptor"] = snomed["CPT Descriptor"].astype('str')
snomed.loc[snomed["CPT Descriptor"] == "nan", "CPT Descriptor"] = ""

snomed

## Generate Mappings

In [ ]:
snomed.loc[:, "pk"] = "CONCEPT:" + snomed["Concept Id"]

snomed.loc[snomed["Map Category"] == "Unmappable", "sk"] = "UNMAPPABLE:" + snomed.loc[snomed["Map Category"] == "Unmappable", "Map Id"].astype('str')
snomed.loc[~(snomed["Map Category"] == "Unmappable"), "sk"] = "CPT:" + snomed.loc[~(snomed["Map Category"] == "CPT"), "CPT Code"].astype(str)
snomed = snomed.drop_duplicates(subset=("pk", "sk"))

mappings = snomed.loc[:, ["sk", "pk", "FSN", "Map Category", "CPT Descriptor"]].rename(
    columns={
        "FSN": "snomed_descriptor",
        "Map Category": "map_category",
        "CPT Descriptor": "cpt_descriptor",
    }
)

mappings

## Generate Keyword Map

In [ ]:
mappings["keyword"] = mappings.snomed_descriptor.apply(lambda x: re.sub(r'[^\w ]+', '', x)).str.lower().str.split()
keyword_map = mappings.loc[:, ["pk", "sk", "keyword"]].explode("keyword").reset_index(drop=True).drop_duplicates()

mappings = mappings.drop(columns="keyword")

keyword_map

In [ ]:
keyword_map[keyword_map.pk == "CONCEPT:609038002"]

## Generate Mapping and Keyword Table Items

In [ ]:
items = []

for index in range(len(mappings)):
    row = mappings.iloc[index]

    items.append(row.to_dict())

items += [dict(pk=f"{row.pk}:{row.sk}", sk=f"KEYWORD:{row.keyword}") for index, row in keyword_map.iterrows() if row.sk.startswith("CPT:")]

items

In [ ]:
print(len(items))
items[-10:]